In [1]:
import pandas as pd
from datetime import datetime

def parse_datetime(date_string):
    """特殊な日時形式をパースする"""
    return datetime.strptime(date_string.strip(), "%B %d %Y @ %I:%M:%S %p")

def parse_price(price_str):
    """カンマを含む価格文字列をfloatに変換する"""
    return float(price_str.replace(',', '').replace('$', '').strip())

def read_trade_data(file_path):
    """CSVファイルからトレードデータを読み込み、構造化されたDataFrameに変換する"""
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    trades = []
    for i in range(0, len(lines), 12):  # 12行で1トレード
        if i + 11 < len(lines):
            trade = {
                'id': lines[i].strip(),
                'symbol': lines[i + 1].strip(),
                'size': int(lines[i + 2].strip()),
                'entry_time': parse_datetime(lines[i + 3]),
                'exit_time': parse_datetime(lines[i + 4]),
                'entry_price': parse_price(lines[i + 5]),
                'exit_price': parse_price(lines[i + 6]),
                'pnl': parse_price(lines[i + 7]),
                'commission': parse_price(lines[i + 8]),
                'fees': parse_price(lines[i + 9]),
                'direction': lines[i + 10].strip()
            }
            trades.append(trade)
    
    return pd.DataFrame(trades)

# Jupyternotebookで実行する部分
# %%
# データの読み込み
file_path = "trade.csv"
df = read_trade_data(file_path)

# 基本的な情報の確認
print("=== データの基本情報 ===")
print(f"トレード総数: {len(df)}")
print("\n=== データフレームの先頭5行 ===")
display(df.head())

# 基本的な統計情報
print("\n=== 損益の基本統計 ===")
print(df['pnl'].describe())

# 日付ごとの集計
df['trade_date'] = df['entry_time'].dt.date
daily_summary = df.groupby('trade_date').agg({
    'pnl': ['count', 'sum', 'mean'],
    'size': 'sum'
}).round(2)

print("\n=== 日別サマリー ===")
display(daily_summary)

=== データの基本情報 ===
トレード総数: 68

=== データフレームの先頭5行 ===


,id,symbol,size,entry_time,exit_time,entry_price,exit_price,pnl,commission,fees,direction
0,504886786,/MNQ,5,2025-01-04 01:17:02,2025-01-04 01:19:46,21368.25,21380.00,-117.5,0.0,-3.70,Short
1,504857518,/MNQ,5,2025-01-04 01:10:43,2025-01-04 01:14:32,21345.75,21361.00,-152.5,0.0,-3.70,Short
2,504768524,/MNQ,2,2025-01-04 00:59:38,2025-01-04 01:02:47,21374.75,21380.00,-21.0,0.0,-1.48,Short
3,504769583,/MNQ,3,2025-01-04 00:59:38,2025-01-04 01:02:53,21374.75,21385.25,-63.0,0.0,-2.22,Short
4,504608623,/MNQ,5,2025-01-04 00:33:18,2025-01-04 00:38:24,21340.25,21340.25,0.0,0.0,-3.70,Long



=== 損益の基本統計 ===
count     68.000000
mean      21.558824
std      134.259065
min     -292.500000
25%      -42.125000
50%        0.000000
75%       74.375000
max      568.000000
Name: pnl, dtype: float64

=== 日別サマリー ===


pnl                size
           count    sum    mean  sum
trade_date                          
2024-12-12     5  -40.5   -8.10    5
2024-12-13     5   63.0   12.60    5
2024-12-14     1  292.5  292.50    1
2024-12-17     3  -23.0   -7.67    3
2024-12-19     2  160.0   80.00    2
2024-12-20     2  -40.5  -20.25    3
2024-12-24     3  324.0  108.00    6
2024-12-26     1  299.0  299.00    2
2024-12-28     9  -82.0   -9.11   10
2025-01-01    16 -417.0  -26.06   24
2025-01-02     5  703.0  140.60    5
2025-01-03     6  663.5  110.58   10
2025-01-04    10 -436.0  -43.60   31

In [2]:
import sqlite3
from datetime import datetime

# データベースの作成と接続
conn = sqlite3.connect('trades.db')
cursor = conn.cursor()

# トレードテーブルの作成
cursor.execute("""
CREATE TABLE IF NOT EXISTS trades (
    id INTEGER PRIMARY KEY,
    trade_id TEXT UNIQUE,
    symbol TEXT NOT NULL,
    size INTEGER NOT NULL,
    entry_time TIMESTAMP NOT NULL,
    exit_time TIMESTAMP NOT NULL,
    entry_price REAL NOT NULL,
    exit_price REAL NOT NULL,
    pnl REAL NOT NULL,
    commission REAL NOT NULL,
    fees REAL NOT NULL,
    direction TEXT NOT NULL
)
""")

# DataFrameをSQLiteデータベースに格納
df.to_sql('trades', conn, if_exists='replace', index=False)

# データが正しく格納されたか確認
test_query = pd.read_sql_query("SELECT * FROM trades LIMIT 5", conn)
display(test_query)

# 基本的な統計情報の確認
stats_query = pd.read_sql_query("""
SELECT 
    COUNT(*) as total_trades,
    SUM(pnl) as total_pnl,
    AVG(pnl) as avg_pnl,
    SUM(CASE WHEN pnl > 0 THEN 1 ELSE 0 END) as winning_trades,
    SUM(CASE WHEN pnl < 0 THEN 1 ELSE 0 END) as losing_trades
FROM trades
""", conn)
display(stats_query)

conn.close()

,id,symbol,size,entry_time,exit_time,entry_price,exit_price,pnl,commission,fees,direction,trade_date
0,504886786,/MNQ,5,2025-01-04 01:17:02,2025-01-04 01:19:46,21368.25,21380.00,-117.5,0.0,-3.70,Short,2025-01-04
1,504857518,/MNQ,5,2025-01-04 01:10:43,2025-01-04 01:14:32,21345.75,21361.00,-152.5,0.0,-3.70,Short,2025-01-04
2,504768524,/MNQ,2,2025-01-04 00:59:38,2025-01-04 01:02:47,21374.75,21380.00,-21.0,0.0,-1.48,Short,2025-01-04
3,504769583,/MNQ,3,2025-01-04 00:59:38,2025-01-04 01:02:53,21374.75,21385.25,-63.0,0.0,-2.22,Short,2025-01-04
4,504608623,/MNQ,5,2025-01-04 00:33:18,2025-01-04 00:38:24,21340.25,21340.25,0.0,0.0,-3.70,Long,2025-01-04


,total_trades,total_pnl,avg_pnl,winning_trades,losing_trades
0,68,1466.0,21.558824,33,33
